In [4]:
from google.colab import drive
import sys
import os

drive.mount('/content/drive')

# Personalized path to your project directory
path = '/content/drive/MyDrive/cod/LEA3_health_7' # Replace with your actual path

# Add the path to sys.path
sys.path.append(path)
os.chdir(path)

os.curdir

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'.'

In [8]:
import numpy as np
import pandas as pd
import cv2 ### para leer imagenes jpeg
### pip install opencv-python
from  _funciones import cargar_zip, crear_diccionario, cargar_csv, crear_ruta, crear_binaria, reemplazar_nulos, redimensionar_imagen
import tensorflow as tf
import openpyxl

import sys
sys.executable
sys.path

import numpy as np

if __name__ == "__main__":

    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import FunctionTransformer

    extract_to = cargar_zip(
        '/content/drive/MyDrive/cod/LEA3_health_7/data/HAM10000_images_part_1.zip',
        '/content/drive/MyDrive/cod/LEA3_health_7/data/HAM10000_images_part_2.zip',
        '/content/ham10000_images'
    )

    imageid_path_dict = crear_diccionario(extract_to)
    lesion_type_dict = {}

    skin_df = cargar_csv('/content/drive/MyDrive/cod/LEA3_health_7/data/HAM10000_metadata.csv')

    pipeline_total = Pipeline(steps=[
        ('Ruta', FunctionTransformer(lambda df: crear_ruta(df, imageid_path_dict, lesion_type_dict), validate=False)),
        ('Binaria', FunctionTransformer(crear_binaria, validate=False)),
        ('Nulos', FunctionTransformer(lambda df: reemplazar_nulos(df, columna='age'), validate=False)),
        ('Imagen', FunctionTransformer(lambda df: redimensionar_imagen(df, size=(100, 75)), validate=False))
    ])

    skin_df = pipeline_total.fit_transform(skin_df)

    # Extraer imágenes del df
    x = np.stack(skin_df['image'].values).astype('float32')
    x /= 255.0  # escalar

    # Cargar el modelo ya entrenado
    modelo = tf.keras.models.load_model('salidas/best_cnn_rec.keras')

    # Realizar predicciones
    prob = modelo.predict(x).ravel()

    # Clasificar según umbrales
    clas = [
    'No ident' if prob > 0.05 else
    "No maligno"
    for prob in prob
]

    # Guardar resultados en Excel
    resultados = pd.DataFrame({
        "paciente": skin_df['image_id'],
        "clasificación": clas,
        "probabilidad": prob
    })

    resultados.to_excel('salidas/clasificados_skin.xlsx', index=False)
    print("¡Clasificación completada y guardada!")


313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 76ms/step
¡Clasificación completada y guardada!
